In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [2]:
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

In [3]:
import os

from torch.utils.data import Dataset, DataLoader

from PIL import Image
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# 1. Chuẩn bị dữ liệu

- Bước 1: Đọc ảnh từ ổ cứng
- Bước 2: Biến đổi ảnh
- Bước 3: Tạo label tương ứng với ảnh

In [4]:
class MyBrainTumorDataset(Dataset):
    def __init__(self, data_folder, csv_path):
        self.data_folder = data_folder
        self.image_names = [name for name in os.listdir(data_folder) if name.endswith('.jpg')]

        # Transformation => Bước 2
        self.transform = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        
        # Label => Bước 3
        self.label_df = pd.read_csv(csv_path, usecols=['image_name', 'label'])
        self.image_names = [name for name in self.image_names if name in self.label_df.image_name.to_list()]
        
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]

        # Buoc 1: Doc anh tu o cung
        image = Image.open(os.path.join(self.data_folder, image_name))

        # Buoc 2: Bien doi anh
        transformed_image = self.transform(image)

        # Buoc 3: Tao label cho anh
        label_str = self.label_df[self.label_df.image_name == image_name]['label'].values[0]
        if label_str == 'tumor':
            label = 1
        else:
            label = 0

        return transformed_image, label

In [5]:
train_dataset = MyBrainTumorDataset(
    data_folder='brain_tumor_mri_dataset/train',
    csv_path='brain_tumor_mri_dataset/label.csv'
)

In [6]:
len(train_dataset)

5744

In [7]:
train_dataset[0]

(tensor([[[-0.7333, -0.2784, -0.6706,  ..., -0.7961,  0.0588, -0.5843],
          [-0.7255, -0.4118, -0.6627,  ..., -0.8275, -0.0902, -0.7020],
          [-0.9765, -0.9529, -0.9843,  ..., -0.8353, -0.1608, -0.5373],
          ...,
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922]],
 
         [[-0.7333, -0.2784, -0.6706,  ..., -0.7961,  0.0588, -0.5843],
          [-0.7255, -0.4118, -0.6627,  ..., -0.8275, -0.0902, -0.7020],
          [-0.9765, -0.9529, -0.9843,  ..., -0.8353, -0.1608, -0.5373],
          ...,
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922]],
 
         [[-0.7333, -0.2784, -0.6706,  ..., -0.7961,  0.0588, -0.5843],
          [-0.7255, -0.4118,

In [8]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    num_workers=0
)
train_dataloader

# 2. Xây dựng mô hình

In [9]:
class MyCustomCNN(nn.Module):
    def __init__(self):
        super(MyCustomCNN, self).__init__()

        self.conv_1 = nn.Conv2d(
            in_channels=3,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu_1 = nn.ReLU()
        self.pool_1 = nn.MaxPool2d(
            kernel_size=2,
            stride=2
        )

        self.conv_2 = nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu_2 = nn.ReLU()
        self.pool_2 = nn.MaxPool2d(
            kernel_size=2,
            stride=2
        )

        self.linear_1 = nn.Linear(32 * 16 * 16, 128)
        self.relu_3 = nn.ReLU()

        self.linear_2 = nn.Linear(128, 2)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.conv_1(x)
        x = self.relu_1(x)
        x = self.pool_1(x)

        x = self.conv_2(x)
        x = self.relu_2(x)
        x = self.pool_2(x)
        
        x = x.view(-1, 32 * 16 * 16)
        x = self.linear_1(x)
        x = self.relu_3(x)
        
        x = self.linear_2(x)
        x = self.softmax(x)
        return x

In [10]:
model = MyCustomCNN()

In [11]:
# model.cuda()

In [12]:
model

MyCustomCNN(
  (conv_1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu_1): ReLU()
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu_2): ReLU()
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_1): Linear(in_features=8192, out_features=128, bias=True)
  (relu_3): ReLU()
  (linear_2): Linear(in_features=128, out_features=2, bias=True)
  (softmax): Softmax(dim=None)
)

In [13]:
class MyResNetCNN(nn.Module):
    def __init__(self):
        super(MyResNetCNN, self).__init__()
        
        self.backbone = models.resnet18(pretrained=True)
        num_features = self.backbone.fc.in_features
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-1])
        
        self.linear = nn.Linear(num_features, 2)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.backbone(x)
        x = x.squeeze()
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [14]:
model = MyResNetCNN()
model

/Users/minhhuunguyen/REPOSITORY/minhhuunguyen.github.io/posts/ai-lectures/venv/common_venv/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/minhhuunguyen/REPOSITORY/minhhuunguyen.github.io/posts/ai-lectures/venv/common_venv/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MyResNetCNN(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [15]:
# model.cuda()

# 3. Huấn luyện mô hình

## 3.1. Khởi tạo hàm Loss và thuật toán tối ưu Optimizer

In [16]:
loss_func = nn.CrossEntropyLoss()
loss_func

CrossEntropyLoss()

In [17]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)

## 3.2. Huấn luyện và đánh giá mô hình

In [18]:
writer = SummaryWriter(log_dir='train_logs')
writer

In [19]:
num_epoch = 10

In [20]:
ckpt_dir = 'ckpt'
os.makedirs(ckpt_dir, exist_ok=True)

In [21]:
for epoch in range(num_epoch):
    # Train model
    model.train()
    for iteration_, (image, label) in enumerate(tqdm(train_dataloader, total=len(train_dataloader))):
        
        # Bước 1: Optimizer zero grad
        optimizer.zero_grad()

        # Bước 2: Foward data to model
        # image.cuda()
        pred = model(image)

        # Bước 3: Tính giá trị loss
        loss_value = loss_func(pred, label)

        # Bước 4: Cập nhật trọng số của mô hình
        loss_value.backward()
        optimizer.step()
        
        global_iteration = epoch * len(train_dataloader) + iteration_
        writer.add_scalar('train_loss_iter', loss_value, global_iteration)

    # Bước 5: (Tuỳ chọn) In các thông số ra ngoài màn hình
    print(f'Epoch={epoch}', f'Training loss={loss_value.item()}')
    writer.add_scalar('train_loss_epoch', loss_value, epoch)

    # Evaluate model
    model.eval()
    with torch.no_grad():
        loss_sum = 0
        pred_list, label_list = [], []
        for image, label in tqdm(train_dataloader, total=len(train_dataloader)):
            # image.cuda()
            pred = model(image)
            loss = loss_func(pred, label)
            loss_sum += loss.item()

            pred_list.append(pred)
            label_list.append(label)

        print(f'Test loss {loss_sum / len(train_dataloader)}')
        writer.add_scalar('test_loss_epoch', loss_value, epoch)
        
        # Calculate metrics
        final_pred = torch.concat(pred_list)
        final_pred = torch.argmax(final_pred, axis=1)
        final_label = torch.concat(label_list)
        
        epoch_accuracy_score = accuracy_score(final_pred, final_label)
        writer.add_scalar('test_accuracy_score_epoch', epoch_accuracy_score, epoch)

        epoch_precision_score = precision_score(final_pred, final_label)
        writer.add_scalar('test_precision_score_epoch', epoch_precision_score, epoch)

        epoch_recall_score = recall_score(final_pred, final_label)
        writer.add_scalar('test_recall_score_epoch', epoch_recall_score, epoch)

        epoch_f1_score = f1_score(final_pred, final_label)
        writer.add_scalar('test_f1_score_epoch', epoch_f1_score, epoch)

        print(classification_report(final_pred, final_label))
        
        torch.save(model.state_dict(), os.path.join(ckpt_dir, f'ckpt_{epoch}.pth'))

  0%|                                                                                                                        | 0/2872 [00:00<?, ?it/s]/Users/minhhuunguyen/REPOSITORY/minhhuunguyen.github.io/posts/ai-lectures/venv/common_venv/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2872/2872 [01:33<00:00, 30.61it/s]


Epoch=0 Training loss=0.3132641017436981


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2872/2872 [00:36<00:00, 78.97it/s]


Test loss 0.37830169294496435
              precision    recall  f1-score   support

           0       0.73      0.94      0.82       939
           1       0.99      0.93      0.96      4805

    accuracy                           0.93      5744
   macro avg       0.86      0.93      0.89      5744
weighted avg       0.94      0.93      0.94      5744



  0%|                                                                                                                        | 0/2872 [00:00<?, ?it/s]/Users/minhhuunguyen/REPOSITORY/minhhuunguyen.github.io/posts/ai-lectures/venv/common_venv/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2872/2872 [01:33<00:00, 30.64it/s]


...

Test loss 0.3481675073981783
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1185
           1       0.98      0.97      0.98      4559

    accuracy                           0.96      5744
   macro avg       0.94      0.95      0.95      5744
weighted avg       0.96      0.96      0.96      5744



  0%|                                                                                                                        | 0/2872 [00:00<?, ?it/s]/Users/minhhuunguyen/REPOSITORY/minhhuunguyen.github.io/posts/ai-lectures/venv/common_venv/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2872/2872 [01:27<00:00, 32.91it/s]


Epoch=9 Training loss=0.31327927112579346


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2872/2872 [00:33<00:00, 85.06it/s]


Test loss 0.3415459215973081
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      1207
           1       0.98      0.98      0.98      4537

    accuracy                           0.97      5744
   macro avg       0.96      0.96      0.96      5744
weighted avg       0.97      0.97      0.97      5744



# 4. Predict new data

In [22]:
state_dict = torch.load('ckpt/ckpt_0.pth', map_location='cpu')
state_dict

OrderedDict([('backbone.0.weight',
              tensor([[[[-1.3875e-02, -5.8965e-03, -2.9944e-03,  ...,  4.9242e-02,
                          1.1644e-02, -1.5179e-02],
                        [ 1.4622e-02,  1.6225e-02, -1.0648e-01,  ..., -2.7473e-01,
                         -1.3304e-01,  4.4973e-03],
                        [-3.8930e-03,  6.3580e-02,  2.9982e-01,  ...,  5.1719e-01,
                          2.5064e-01,  5.9347e-02],
                        ...,
                        [-2.7906e-02,  1.6048e-02,  6.9598e-02,  ..., -3.3980e-01,
                         -4.3255e-01, -2.6715e-01],
                        [ 3.3713e-02,  3.5409e-02,  5.8202e-02,  ...,  4.0306e-01,
                          3.8190e-01,  1.5948e-01],
                        [-1.3356e-02, -1.1038e-02, -3.0789e-02,  ..., -1.6188e-01,
                         -9.2309e-02, -1.4061e-02]],
              
                       [[-1.4811e-02, -2.6337e-02, -3.5815e-02,  ...,  2.5188e-02,
                         -4

In [23]:
# new_model_1 = MyCustomCNN()

In [24]:
new_model_2 = MyResNetCNN()

/Users/minhhuunguyen/REPOSITORY/minhhuunguyen.github.io/posts/ai-lectures/venv/common_venv/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/minhhuunguyen/REPOSITORY/minhhuunguyen.github.io/posts/ai-lectures/venv/common_venv/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
new_model_2.load_state_dict(state_dict)

<All keys matched successfully>

In [26]:
# new_model_1.load_state_dict(state_dict)

In [27]:
new_model_2.eval()
with torch.no_grad():
    loss_sum = 0
    pred_list, label_list = [], []
    for image, label in tqdm(train_dataloader, total=len(train_dataloader)):
        pred = new_model_2(image)
        loss = loss_func(pred, label)
        loss_sum += loss.item()

        pred_list.append(pred)
        label_list.append(label)

    print(f'Test loss {loss_sum / len(train_dataloader)}')

    # Calculate metrics
    final_pred = torch.concat(pred_list)
    final_pred = torch.argmax(final_pred, axis=1)
    final_label = torch.concat(label_list)

    print(classification_report(final_pred, final_label))

  0%|                                                                                                                        | 0/2872 [00:00<?, ?it/s]/Users/minhhuunguyen/REPOSITORY/minhhuunguyen.github.io/posts/ai-lectures/venv/common_venv/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2872/2872 [00:33<00:00, 85.23it/s]

Test loss 0.3783016928411958
              precision    recall  f1-score   support

           0       0.73      0.94      0.82       939
           1       0.99      0.93      0.96      4805

    accuracy                           0.93      5744
   macro avg       0.86      0.93      0.89      5744
weighted avg       0.94      0.93      0.94      5744

